# Anndata介绍

`AnnData`（全称 `Annotated Data`）是 Scanpy 的核心数据结构，可以理解成 Python 版的 `SingleCellExperiment`。
它是为矩阵型数据设计的，特点是：
.行（`obs`）：观测值（单细胞分析里 = 细胞/样本），通常有唯一的索引（`barcode`）
.列（`var`）：特征（单细胞分析里 = 基因/峰/蛋白），也有唯一的索引（`gene ID`）
.`.obs` 和 `.var`：行/列的元数据（`metadata`）
.`uns`：不规则的、结构化或非结构化的元信息（如颜色、聚类结果、分析参数）
.`X`：核心数据矩阵（表达量、计数等），可以是稠密矩阵（`NumPy array`）或稀疏矩阵（`scipy.sparse`）
.支持稀疏矩阵（非常重要，因为单细胞数据大部分是 0）

类比：
在 R/Seurat 里，`.X` 类似 `counts(sce)` 或 `GetAssayData(seurat, slot="counts")`
`.obs` 类似 `colData(sce)`
`.var` 类似 `rowData(sce)`
`.uns` 类似 `metadata(sce)` 或 `Seurat 的 @misc`

## 下面创建一个anndata对象，以随机生成的稀疏矩阵为例：

In [1]:
import numpy as np
import pandas as pd
import anndata as ad
from scipy.sparse import csr_matrix
print(ad.__version__)

0.11.4


In [2]:
## 创建一个100x2000的稀疏矩阵来表征基因表达数据
counts = csr_matrix(np.random.poisson(1, size=(100, 2000)), dtype=np.float32)
adata = ad.AnnData(counts)
adata

AnnData object with n_obs × n_vars = 100 × 2000

In [3]:
## 访问表达矩阵
adata.X

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 126209 stored elements and shape (100, 2000)>

然后，给细胞和基因打上索引（命名）

In [4]:
adata.obs_names = [f"Cell_{i:d}" for i in range(adata.n_obs)]
adata.var_names = [f"Gene_{i:d}" for i in range(adata.n_vars)]
print(adata.obs_names[:10])
print(adata.var_names[:10])

Index(['Cell_0', 'Cell_1', 'Cell_2', 'Cell_3', 'Cell_4', 'Cell_5', 'Cell_6',
       'Cell_7', 'Cell_8', 'Cell_9'],
      dtype='object')
Index(['Gene_0', 'Gene_1', 'Gene_2', 'Gene_3', 'Gene_4', 'Gene_5', 'Gene_6',
       'Gene_7', 'Gene_8', 'Gene_9'],
      dtype='object')


## 筛选感兴趣的细胞和基因

In [5]:
## 筛选成一个新的anndata对象
adata[["Cell_1", "Cell_10"], ["Gene_5", "Gene_1900"]]

View of AnnData object with n_obs × n_vars = 2 × 2

## 增加metadata`adata.obs, adata.var`
在anndata中，细胞和基因的metadata可以用`adata.obs`和`adata.var`来访问，他们本质上都是一个pd.DataFrame

In [6]:
ct = np.random.choice(["B", "T", "Monocyte"], size = (adata.n_obs, ))
adata.obs["cell_type"] = pd.Categorical(ct)
adata.obs

,cell_type
Cell_0,T
Cell_1,Monocyte
Cell_2,Monocyte
Cell_3,T
Cell_4,T
...,...
Cell_95,T
Cell_96,Monocyte
Cell_97,T
Cell_98,B


In [7]:
## 查看更新后的adata信息
adata

AnnData object with n_obs × n_vars = 100 × 2000
    obs: 'cell_type'

可以根据新生成的细胞类型来筛选数据：

In [8]:
bdata = adata[adata.obs.cell_type == "B"]
bdata

View of AnnData object with n_obs × n_vars = 38 × 2000
    obs: 'cell_type'

## 多维元数据的存储方式`.obsm, .varm`
有些元数据不是单一值，而是一个矩阵，比如：
对细胞（observation）来说：`UMAP` 或 `PCA` 降维后的二维坐标
对基因（variable）来说：某个算法计算的多维特征向量
这类数据在 `AnnData` 里不能放在 `.obs` 或 `.var`（因为它们只支持表格型、列状数据），而是放在 `.obsm` / `.varm`。

### `.obsm` 和 `.varm` 的规则
`.obsm（Observation Multi-dimensional）`:
存放细胞层面的多维矩阵
行数必须等于细胞数`n_obs`
比如 `adata.obsm["X_umap"]` 的形状是 (细胞数, 降维维度数)，如 (100, 2)

`.varm（Variable Multi-dimensional）`:
存放基因层面的多维矩阵
行数必须等于基因数`n_vars`
比如 `adata.varm["gene_stuff"]` 的形状是 (基因数, 特征数)，如 (2000, 5)

In [9]:
adata.obsm["X_umap"] = np.random.normal(0, 1, size = (adata.n_obs, 2))
adata.varm["gene_stuff"] = np.random.normal(0, 1, size = (adata.n_vars, 5))
adata.obsm

AxisArrays with keys: X_umap

In [10]:
adata

AnnData object with n_obs × n_vars = 100 × 2000
    obs: 'cell_type'
    obsm: 'X_umap'
    varm: 'gene_stuff'

## 非结构化的元数据`.uns`
`.uns（Unstructured）`是一个 字典（dict-like）容器，可以放任何类型的 Python 对象：数字、列表、字典、颜色映射、描述信息等。

In [11]:
adata.uns["random"] = [1, 2, 3]
adata.uns

OrderedDict([('random', [1, 2, 3])])

## 存放表达数据的不同方式`.layers`
对原始表达矩阵进行不同处理之后，比如标准化后的矩阵，存放在`adata.layers`中。
`adata.X`一般指正在分析的数据，如果用`scanpy`去标准化，会覆盖，所以一般会把原始矩阵copy一份放在`adata.layers["counts"]`里

In [12]:
adata.layers["log_transformed"] = np.log1p(adata.X)

## 通常会把原始矩阵留一份放在layers中
adata.layers["counts"] = adata.X.copy()

adata

AnnData object with n_obs × n_vars = 100 × 2000
    obs: 'cell_type'
    uns: 'random'
    obsm: 'X_umap'
    varm: 'gene_stuff'
    layers: 'log_transformed', 'counts'

同时，还可以将`layers`中的表达矩阵转成`DataFrames`

In [13]:
adata.to_df(layer = "log_transformed").head()

,Gene_0,Gene_1,Gene_2,Gene_3,Gene_4,Gene_5,Gene_6,Gene_7,Gene_8,Gene_9,...,Gene_1990,Gene_1991,Gene_1992,Gene_1993,Gene_1994,Gene_1995,Gene_1996,Gene_1997,Gene_1998,Gene_1999
Cell_0,0.693147,0.000000,1.386294,0.000000,0.000000,0.000000,0.693147,1.098612,0.000000,0.693147,...,0.693147,0.693147,0.693147,0.000000,0.000000,0.000000,0.000000,1.386294,0.000000,1.098612
Cell_1,0.693147,0.000000,1.098612,0.000000,0.693147,0.693147,1.098612,0.693147,0.000000,0.693147,...,0.000000,0.000000,0.000000,0.693147,0.693147,0.000000,0.000000,0.693147,0.000000,0.000000
Cell_2,0.693147,0.000000,0.000000,0.693147,0.693147,0.000000,0.693147,0.000000,0.693147,1.386294,...,1.098612,1.098612,1.098612,0.693147,0.000000,0.693147,0.000000,0.000000,0.693147,0.000000
Cell_3,0.693147,0.693147,1.386294,0.693147,1.609438,1.098612,0.000000,0.000000,1.791759,0.000000,...,0.000000,0.693147,1.098612,0.693147,1.098612,0.693147,0.693147,0.693147,0.693147,0.693147
Cell_4,1.098612,0.693147,0.693147,0.693147,1.098612,0.693147,0.000000,0.000000,0.693147,0.693147,...,0.000000,0.000000,0.693147,1.386294,0.693147,0.693147,0.000000,1.098612,0.693147,0.693147


## 保存anndata
一般会把`anndata`保存成一个`.h5ad`文件。
`.h5ad`文件是基于 `HDF5` 的二进制文件格式，适合存储矩阵 + 元数据，压缩后体积小、读取快。

`.h5ad` 文件就像是 `AnnData` 对象的“完整快照”，保存了：
主数据矩阵（.X）
所有版本矩阵（.layers）
细胞/基因元数据（.obs / .var）
多维嵌入（.obsm / .varm）
图结构（.obsp / .varp）
额外参数配置（.uns）

In [14]:
adata.write("../data/my_results.h5ad", compression="gzip")

## views and copies

In [15]:
## 创建一个关于细胞的元数据
obs_meta = pd.DataFrame({
        'time_yr': np.random.choice([0, 2, 4, 8], adata.n_obs),
        'subject_id': np.random.choice(['subject 1', 'subject 2', 'subject 4', 'subject 8'], adata.n_obs),
        'instrument_type': np.random.choice(['type a', 'type b'], adata.n_obs),
        'site': np.random.choice(['site x', 'site y'], adata.n_obs),
    },
    index=adata.obs.index,    # 细胞的索引不变
)

In [16]:
## 创建新的anndata对象（使用新的细胞metadata）
adata = ad.AnnData(adata.X, obs=obs_meta, var=adata.var)

In [17]:
print(adata)

AnnData object with n_obs × n_vars = 100 × 2000
    obs: 'time_yr', 'subject_id', 'instrument_type', 'site'


### AnnData 的 View 与 Copy 机制

- **两种状态**  
  - **Actual data**：对象自身持有全部数据。  
  - **View**：仅引用另一个 AnnData 的部分数据，没有独立存储。

- **切片默认返回 View**  
  - 节省内存（不复制数据）。  
  - 可以直接修改原对象的内容。

- **从 View 转换为独立对象**  
  - 显式调用 `.copy()`。  
  - 或者，当对 View 的内容（如 `.obs`、`.var`、`.X`）进行赋值修改时，会自动触发 `.copy()`。

- **优点**  
  1. 高效的内存利用。  
  2. 快速访问和修改原对象的子集数据。

- **注意**  
  如果只是读取子集数据，不会触发 `.copy()`；但一旦修改，就会变成独立数据对象。


In [18]:
## 这是一个view
adata[:5, ['Gene_1', 'Gene_3']]

View of AnnData object with n_obs × n_vars = 5 × 2
    obs: 'time_yr', 'subject_id', 'instrument_type', 'site'

In [19]:
## copy
adata_subset = adata[:5, ['Gene_1', 'Gene_3']].copy()

In [20]:
## 对adata进行修改
print(adata[:3, 'Gene_1'].X.toarray().tolist())
adata[:3, 'Gene_1'].X = [0, 0, 0]
print(adata[:3, 'Gene_1'].X.toarray().tolist())

[[0.0], [0.0], [0.0]]
[[0.0], [0.0], [0.0]]


/var/folders/s8/0mvfbk_j3_7chd0wj4khg3bh0000gn/T/ipykernel_56176/257090664.py:3: ImplicitModificationWarning: Modifying `X` on a view results in data being overridden
  adata[:3, 'Gene_1'].X = [0, 0, 0]


In [21]:
## 自动创建对象
adata_subset = adata[:3, ['Gene_1', 'Gene_2']]
adata_subset

View of AnnData object with n_obs × n_vars = 3 × 2
    obs: 'time_yr', 'subject_id', 'instrument_type', 'site'

In [22]:
adata_subset.obs['foo'] = range(3)
adata_subset

/var/folders/s8/0mvfbk_j3_7chd0wj4khg3bh0000gn/T/ipykernel_56176/4201540124.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_subset.obs['foo'] = range(3)


AnnData object with n_obs × n_vars = 3 × 2
    obs: 'time_yr', 'subject_id', 'instrument_type', 'site', 'foo'

In [23]:
## 做类似于pandas对切片操作
# 判断time_yr在2和4的细胞
adata[adata.obs.time_yr.isin([2, 4])].obs.head()

,time_yr,subject_id,instrument_type,site
Cell_1,4,subject 8,type a,site y
Cell_2,2,subject 8,type b,site x
Cell_4,2,subject 2,type a,site y
Cell_5,4,subject 2,type b,site y
Cell_6,4,subject 1,type b,site y


## `.h5ad`文件的部分读取
`backed='r'`表示只读模式下的部分加载。AnnData不会一次性把所有数据读到内存，而是保留一个到磁盘文件的连接。
当访问`.X `的部分数据时，才会从磁盘读入这一部分（按需读取）。

In [24]:
adata = ad.read('../data/my_results.h5ad', backed='r')
adata.isbacked

/opt/anaconda3/envs/scepy/lib/python3.10/site-packages/anndata/__init__.py:42: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


True

In [25]:
adata.filename

PosixPath('../data/my_results.h5ad')

In [26]:
adata.file.close()